In [1]:
import requests
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline
from scipy.stats import norm
from matplotlib.backends.backend_pdf import PdfPages
from datetime import datetime

BASE_URL = "https://www.deribit.com/api/v2"

def convert_timestamp_to_datetime(timestamp_ms):
    """Convert timestamp in milliseconds to readable datetime format"""
    timestamp_s = timestamp_ms / 1000  # Convert milliseconds to seconds
    dt = datetime.fromtimestamp(timestamp_s)
    return dt.strftime("%Y-%m-%d %H:%M:%S")

def numerical_derivative(func, x, h=1e-5, n=1):
    """Compute numerical derivative of function at point x"""
    if n == 1:
        return (func(x + h) - func(x - h)) / (2 * h)
    elif n == 2:
        return (func(x + h) - 2 * func(x) + func(x - h)) / (h ** 2)
    else:
        raise ValueError("Only first and second derivatives supported")

def get_spot_price(currency="BTC"):
    """Get current spot price"""
    url = f"{BASE_URL}/public/ticker"
    params = {"instrument_name": f"{currency}-PERPETUAL"}
    
    response = requests.get(url, params=params)
    response.raise_for_status()
    data = response.json()
    
    return data["result"]["mark_price"]

def get_options_data(currency="BTC"):
    """Fetch all options instruments for a currency"""
    url = f"{BASE_URL}/public/get_instruments"
    params = {
        "currency": currency,
        "kind": "option"
    }
    
    response = requests.get(url, params=params)
    response.raise_for_status()
    data = response.json()
    
    # Filter out expired options
    active_instruments = [inst for inst in data["result"] if not inst.get("is_active", True) is False]
    
    return active_instruments

def get_orderbook(instrument_name):
    """Get orderbook for specific instrument"""
    url = f"{BASE_URL}/public/get_order_book"
    params = {
        "instrument_name": instrument_name
    }
    
    response = requests.get(url, params=params)
    response.raise_for_status()
    data = response.json()
    
    return data["result"]

def get_option_data_for_expiry(expiry_timestamp, currency="BTC"):
    """Get strikes and IVs for a specific expiry"""
    instruments = get_options_data(currency)
    
    # Filter for specific expiry and calls only
    expiry_options = [i for i in instruments 
                     if i['expiration_timestamp'] == expiry_timestamp 
                     and i['option_type'] == 'call']
    
    strikes = []
    ivs = []
    
    for option in expiry_options:
        try:
            orderbook = get_orderbook(option['instrument_name'])
            if orderbook.get('mark_iv') is not None:
                strikes.append(option['strike'])
                ivs.append(orderbook['mark_iv'] / 100)  # Convert to decimal
        except:
            continue
    
    # Sort by strike
    if strikes and ivs:
        sorted_data = sorted(zip(strikes, ivs))
        strikes, ivs = zip(*sorted_data)
    
    return np.array(strikes), np.array(ivs)

def black_scholes_call(S, K, T, r, sigma):
    """Black-Scholes call option price"""
    d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)

def breeden_litzenberger(strikes, ivs, spot, r=0.0, T=0.25):
    """Apply Breeden-Litzenberger to extract risk-neutral density"""
    
    # Need minimum points for interpolation
    if len(strikes) < 4:
        raise ValueError(f"Need at least 4 data points, got {len(strikes)}")
    
    # Interpolate IV smile
    iv_spline = CubicSpline(strikes, ivs, bc_type='natural')
    
    # Generate dense strike grid - STAY WITHIN MARKET DATA BOUNDS
    K_min, K_max = strikes.min(), strikes.max()  # No extrapolation!
    K_dense = np.linspace(K_min, K_max, 200)
    iv_dense = iv_spline(K_dense)
    
    # Ensure IV stays positive (clamp negative values)
    iv_dense = np.maximum(iv_dense, 0.01)  # Minimum 1% IV
    
    # Calculate call prices using Black-Scholes
    call_prices = [black_scholes_call(spot, K, T, r, iv) for K, iv in zip(K_dense, iv_dense)]
    
    # Create call price spline
    call_spline = CubicSpline(K_dense, call_prices, bc_type='natural')
    
    # Extract risk-neutral density (second derivative)
    rn_density = [numerical_derivative(call_spline, k, h=1.0, n=2) for k in K_dense]
    rn_density = np.maximum(rn_density, 0) * np.exp(r*T)  # Ensure non-negative
    
    # Normalize to make it a proper probability density
    if np.sum(rn_density) > 0:
        rn_density = rn_density / (np.sum(rn_density) * (K_dense[1] - K_dense[0]))
    
    return K_dense, np.array(rn_density)

def main():
    """Main execution function - Loop through all expiries"""
    
    # Get current spot price
    print("Fetching spot price...")
    spot = get_spot_price("BTC")
    print(f"BTC Spot Price: ${spot:.2f}")
    
    # Get available options
    print("Fetching options data...")
    instruments = get_options_data("BTC")
    
    if not instruments:
        raise Exception("No options instruments found")
    
    # Get unique expiries
    expiries = sorted(list(set([i['expiration_timestamp'] for i in instruments])))
    print(f"Found {len(expiries)} expiry dates")
    
    if not expiries:
        raise Exception("No expiry dates found")
    
    # Print all available expiries in readable format
    print("\nAvailable expiries:")
    for i, expiry in enumerate(expiries):
        expiry_datetime = convert_timestamp_to_datetime(expiry)
        print(f"  {i+1}. {expiry_datetime}")
    
    # Storage for all results
    all_results = []
    
    # Create separate PDFs
    iv_smiles_pdf = "iv_smiles_analysis.pdf"
    rn_distributions_pdf = "risk_neutral_distributions.pdf"
    
    with PdfPages(iv_smiles_pdf) as iv_pdf, PdfPages(rn_distributions_pdf) as rn_pdf:
        
        # Loop through all expiries (limit to first 10 for performance)
        for i, expiry in enumerate(expiries[:10]):
            try:
                expiry_str = convert_timestamp_to_datetime(expiry)
                print(f"\nProcessing expiry {i+1}/{min(10, len(expiries))}: {expiry_str}")
                
                # Get option data for this expiry
                strikes, ivs = get_option_data_for_expiry(expiry)
                print(f"  Found {len(strikes)} options with IV data")
                
                if len(strikes) < 4:
                    print(f"  Skipping - insufficient data points: {len(strikes)}")
                    continue
                
                # Apply Breeden-Litzenberger
                print("  Applying Breeden-Litzenberger...")
                K_dense, rn_density = breeden_litzenberger(strikes, ivs, spot)
                
                # Store results
                result = {
                    'expiry': expiry,
                    'expiry_str': expiry_str,
                    'strikes': strikes,
                    'ivs': ivs,
                    'K_dense': K_dense,
                    'rn_density': rn_density,
                    'spot': spot
                }
                all_results.append(result)
                
                # Create IV Smile plot
                fig, ax = plt.subplots(1, 1, figsize=(10, 6))
                ax.scatter(strikes, ivs*100, color='blue', s=50, label='Market IVs', alpha=0.7)
                iv_interp = CubicSpline(strikes, ivs, bc_type='natural')(K_dense)
                ax.plot(K_dense, iv_interp*100, 'r-', alpha=0.8, label='Interpolated', linewidth=2)
                ax.axvline(spot, color='green', linestyle='--', label=f'Spot: {spot:.0f}', linewidth=2)
                ax.set_xlabel('Strike Price')
                ax.set_ylabel('Implied Volatility (%)')
                ax.set_title(f'IV Smile - Expiry: {expiry_str}')
                ax.legend()
                ax.grid(True, alpha=0.3)
                plt.tight_layout()
                iv_pdf.savefig(fig, dpi=300, bbox_inches='tight')
                plt.close()
                
                # Create Risk-Neutral Density plot
                fig, ax = plt.subplots(1, 1, figsize=(10, 6))
                ax.plot(K_dense, rn_density, 'b-', linewidth=2, label='RN Density')
                ax.fill_between(K_dense, rn_density, alpha=0.3, color='lightblue')
                ax.axvline(spot, color='green', linestyle='--', label=f'Spot: {spot:.0f}', linewidth=2)
                ax.set_xlabel('Strike Price')
                ax.set_ylabel('Probability Density')
                ax.set_title(f'Risk-Neutral Distribution - Expiry: {expiry_str}')
                ax.legend()
                ax.grid(True, alpha=0.3)
                plt.tight_layout()
                rn_pdf.savefig(fig, dpi=300, bbox_inches='tight')
                plt.close()
                
                print(f"  ✅ Processed successfully")
                
            except Exception as e:
                print(f"  ❌ Error processing expiry {expiry_str}: {e}")
                continue
    
    print(f"\n✅ Analysis complete!")
    print(f"📊 Successfully processed {len(all_results)} expiries")
    print(f"📄 IV Smiles saved to: {iv_smiles_pdf}")
    print(f"📄 Risk-Neutral Distributions saved to: {rn_distributions_pdf}")
    
    # Print summary
    if all_results:
        print(f"\nSummary:")
        for result in all_results:
            print(f"- {result['expiry_str']}: {len(result['strikes'])} strikes, "
                  f"IV range: {result['ivs'].min()*100:.1f}%-{result['ivs'].max()*100:.1f}%, "
                  f"Peak: ${result['K_dense'][np.argmax(result['rn_density'])]:.0f}")
    
    # Store results globally for dashboard
    global analysis_results
    analysis_results = all_results
    
    return all_results

if __name__ == "__main__":
    results = main()

Fetching spot price...
BTC Spot Price: $113455.47
Fetching options data...
BTC Spot Price: $113455.47
Fetching options data...
Found 12 expiry dates

Available expiries:
  1. 2025-08-22 10:00:00
  2. 2025-08-23 10:00:00
  3. 2025-08-24 10:00:00
  4. 2025-08-25 10:00:00
  5. 2025-08-29 10:00:00
  6. 2025-09-05 10:00:00
  7. 2025-09-12 10:00:00
  8. 2025-09-26 10:00:00
  9. 2025-10-31 09:00:00
  10. 2025-12-26 09:00:00
  11. 2026-03-27 09:00:00
  12. 2026-06-26 10:00:00

Processing expiry 1/10: 2025-08-22 10:00:00
Found 12 expiry dates

Available expiries:
  1. 2025-08-22 10:00:00
  2. 2025-08-23 10:00:00
  3. 2025-08-24 10:00:00
  4. 2025-08-25 10:00:00
  5. 2025-08-29 10:00:00
  6. 2025-09-05 10:00:00
  7. 2025-09-12 10:00:00
  8. 2025-09-26 10:00:00
  9. 2025-10-31 09:00:00
  10. 2025-12-26 09:00:00
  11. 2026-03-27 09:00:00
  12. 2026-06-26 10:00:00

Processing expiry 1/10: 2025-08-22 10:00:00
  Found 38 options with IV data
  Applying Breeden-Litzenberger...
  Found 38 options with 

# Options Risk-Neutral Density Analysis

This notebook implements the **Breeden-Litzenberger** methodology to extract risk-neutral probability distributions from options market data.

## Key Components:

1. **Data Collection**: Fetches options data from Deribit API
   - Gets implied volatilities for different strikes
   - Focuses on call options for a specific expiry

2. **Breeden-Litzenberger Algorithm**:
   - Interpolates the implied volatility smile
   - Converts IV to Black-Scholes call prices
   - Computes the second derivative of call prices with respect to strike
   - This gives the risk-neutral probability density

3. **Visualizations**:
   - IV smile plot showing market volatilities vs strikes
   - Risk-neutral density distribution
   - 3D visualization of the density surface

## Mathematical Foundation:
The Breeden-Litzenberger formula states that:
```
f(K) = e^(rT) * ∂²C/∂K²
```
Where:
- f(K) is the risk-neutral density at strike K
- C is the call option price
- r is the risk-free rate
- T is time to expiry

## Applications:
- Market sentiment analysis
- Tail risk assessment
- Model validation
- Volatility surface construction

In [4]:
# Interactive Plotly Dashboard
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

def create_interactive_dashboard(results):
    """Create interactive Plotly dashboard for options analysis"""
    
    if not results:
        print("No analysis results available. Please run the main analysis first.")
        return
    
    print(f"Creating interactive dashboard with {len(results)} expiries...")
    
    # Create subplot with secondary y-axis
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('IV Smiles (All Expiries)', 'Risk-Neutral Densities', 
                       'IV Surface (3D)', 'Summary Statistics'),
        specs=[[{'type': 'scatter'}, {'type': 'scatter'}],
               [{'type': 'surface'}, {'type': 'bar'}]],
        vertical_spacing=0.12,
        horizontal_spacing=0.08
    )
    
    # Color palette for different expiries
    colors = px.colors.qualitative.Plotly
    
    # 1. IV Smiles for all expiries
    for i, result in enumerate(results):
        color = colors[i % len(colors)]
        
        # Market IV points
        fig.add_trace(
            go.Scatter(
                x=result['strikes'],
                y=result['ivs']*100,
                mode='markers',
                name=f"Market IV - {result['expiry_str'][:10]}",
                marker=dict(color=color, size=6),
                showlegend=True,
                hovertemplate=f"Expiry: {result['expiry_str']}<br>" +
                            "Strike: %{x:.0f}<br>" +
                            "IV: %{y:.1f}%<extra></extra>"
            ),
            row=1, col=1
        )
        
        # Interpolated IV curve
        iv_interp = CubicSpline(result['strikes'], result['ivs'], bc_type='natural')(result['K_dense'])
        fig.add_trace(
            go.Scatter(
                x=result['K_dense'],
                y=iv_interp*100,
                mode='lines',
                name=f"IV Curve - {result['expiry_str'][:10]}",
                line=dict(color=color, width=2),
                showlegend=False,
                hovertemplate=f"Expiry: {result['expiry_str']}<br>" +
                            "Strike: %{x:.0f}<br>" +
                            "IV: %{y:.1f}%<extra></extra>"
            ),
            row=1, col=1
        )
    
    # Add spot price line to IV plot
    if results:
        spot = results[0]['spot']
        fig.add_vline(x=spot, line_dash="dash", line_color="green", 
                     annotation_text=f"Spot: ${spot:.0f}", row=1, col=1)
    
    # 2. Risk-Neutral Densities
    for i, result in enumerate(results):
        color = colors[i % len(colors)]
        
        fig.add_trace(
            go.Scatter(
                x=result['K_dense'],
                y=result['rn_density'],
                mode='lines',
                name=f"RN Density - {result['expiry_str'][:10]}",
                line=dict(color=color, width=2),
                fill='tonexty' if i > 0 else 'tozeroy',
                fillcolor=f"rgba{tuple(list(px.colors.hex_to_rgb(color)) + [0.1])}",
                hovertemplate=f"Expiry: {result['expiry_str']}<br>" +
                            "Strike: %{x:.0f}<br>" +
                            "Density: %{y:.6f}<extra></extra>"
            ),
            row=1, col=2
        )
    
    # Add spot price line to density plot
    if results:
        fig.add_vline(x=spot, line_dash="dash", line_color="green", 
                     annotation_text=f"Spot: ${spot:.0f}", row=1, col=2)
    
    # 3. IV Surface (3D)
    if len(results) >= 2:
        # Prepare data for 3D surface
        expiry_days = []
        strike_grid = []
        iv_grid = []
        
        # Get common strike range
        min_strike = min([r['strikes'].min() for r in results])
        max_strike = max([r['strikes'].max() for r in results])
        common_strikes = np.linspace(min_strike, max_strike, 50)
        
        for result in results:
            # Calculate days to expiry (simplified)
            expiry_day = (result['expiry'] - results[0]['expiry']) / (1000 * 60 * 60 * 24)
            expiry_days.append(expiry_day)
            
            # Interpolate IV for common strikes
            iv_spline = CubicSpline(result['strikes'], result['ivs'], bc_type='natural')
            iv_values = iv_spline(common_strikes) * 100
            
            strike_grid.append(common_strikes)
            iv_grid.append(iv_values)
        
        # Convert to meshgrid format
        X, Y = np.meshgrid(common_strikes, expiry_days)
        Z = np.array(iv_grid)
        
        fig.add_trace(
            go.Surface(
                x=X,
                y=Y,
                z=Z,
                colorscale='Viridis',
                name='IV Surface',
                hovertemplate="Strike: %{x:.0f}<br>" +
                            "Days to Expiry: %{y:.1f}<br>" +
                            "IV: %{z:.1f}%<extra></extra>"
            ),
            row=2, col=1
        )
    
    # 4. Summary Statistics
    expiry_labels = [r['expiry_str'][:10] for r in results]
    avg_ivs = [r['ivs'].mean() * 100 for r in results]
    iv_ranges = [r['ivs'].std() * 100 for r in results]
    
    fig.add_trace(
        go.Bar(
            x=expiry_labels,
            y=avg_ivs,
            name='Avg IV',
            marker_color='lightblue',
            hovertemplate="Expiry: %{x}<br>" +
                        "Avg IV: %{y:.1f}%<extra></extra>"
        ),
        row=2, col=2
    )
    
    fig.add_trace(
        go.Bar(
            x=expiry_labels,
            y=iv_ranges,
            name='IV Std Dev',
            marker_color='orange',
            yaxis='y2',
            hovertemplate="Expiry: %{x}<br>" +
                        "IV Std: %{y:.1f}%<extra></extra>"
        ),
        row=2, col=2
    )
    
    # Update layout
    fig.update_layout(
        height=1000,
        title_text=f"Interactive Options Analysis Dashboard - {len(results)} Expiries",
        showlegend=True,
        legend=dict(x=1.05, y=1),
        font=dict(size=12)
    )
    
    # Update subplot titles and axes
    fig.update_xaxes(title_text="Strike Price", row=1, col=1)
    fig.update_yaxes(title_text="Implied Volatility (%)", row=1, col=1)
    
    fig.update_xaxes(title_text="Strike Price", row=1, col=2)
    fig.update_yaxes(title_text="Risk-Neutral Density", row=1, col=2)
    
    if len(results) >= 2:
        fig.update_layout(scene=dict(
            xaxis_title="Strike Price",
            yaxis_title="Days to Expiry",
            zaxis_title="Implied Volatility (%)"
        ))
    
    fig.update_xaxes(title_text="Expiry", row=2, col=2)
    fig.update_yaxes(title_text="Average IV (%)", row=2, col=2)
    
    return fig

# Create and display dashboard
if 'analysis_results' in globals():
    dashboard = create_interactive_dashboard(analysis_results)
    dashboard.show()
    
    # Additional analysis: Create volatility term structure
    print("\n📊 Creating Volatility Term Structure...")
    
    fig_term = go.Figure()
    
    for result in analysis_results:
        # Calculate ATM IV (closest to spot)
        spot = result['spot']
        atm_idx = np.argmin(np.abs(result['strikes'] - spot))
        atm_iv = result['ivs'][atm_idx] * 100
        
        # Calculate days to expiry
        days_to_expiry = (result['expiry'] - analysis_results[0]['expiry']) / (1000 * 60 * 60 * 24)
        
        fig_term.add_trace(go.Scatter(
            x=[days_to_expiry],
            y=[atm_iv],
            mode='markers+text',
            marker=dict(size=10, color='blue'),
            text=[result['expiry_str'][:10]],
            textposition='top center',
            name=f"ATM IV - {result['expiry_str'][:10]}",
            hovertemplate=f"Expiry: {result['expiry_str']}<br>" +
                        f"Days to Expiry: {days_to_expiry:.1f}<br>" +
                        f"ATM IV: {atm_iv:.1f}%<extra></extra>"
        ))
    
    fig_term.update_layout(
        title="Volatility Term Structure (ATM)",
        xaxis_title="Days to Expiry",
        yaxis_title="Implied Volatility (%)",
        height=500,
        showlegend=False
    )
    
    fig_term.show()
    
else:
    print("⚠️ No analysis results found. Please run the main analysis cell first.")

Creating interactive dashboard with 10 expiries...



📊 Creating Volatility Term Structure...


In [6]:
# Volatility Surface Construction from Daily Distributions
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

def create_volatility_surface(results):
    """Create comprehensive volatility surface from daily PDF distributions"""
    
    if not results or len(results) < 2:
        print("Need at least 2 expiries to create a surface. Please run the main analysis first.")
        return None
    
    print(f"🏗️ Constructing volatility surface from {len(results)} expiries...")
    
    # Extract data for surface construction
    surface_data = []
    
    for result in results:
        expiry_str = result['expiry_str']
        strikes = result['strikes']
        ivs = result['ivs']
        spot = result['spot']
        
        # Calculate days to expiry
        days_to_expiry = (result['expiry'] - results[0]['expiry']) / (1000 * 60 * 60 * 24)
        
        # Calculate moneyness (Strike/Spot)
        moneyness = strikes / spot
        
        for strike, iv, money in zip(strikes, ivs, moneyness):
            surface_data.append({
                'expiry_str': expiry_str,
                'days_to_expiry': days_to_expiry,
                'strike': strike,
                'iv': iv * 100,  # Convert to percentage
                'moneyness': money,
                'spot': spot
            })
    
    # Convert to DataFrame for easier manipulation
    df = pd.DataFrame(surface_data)
    
    print(f"📊 Surface data: {len(df)} points across {len(results)} expiries")
    
    # Create meshgrid for surface plot
    # Use common strike range across all expiries
    min_strike = df['strike'].min()
    max_strike = df['strike'].max()
    min_days = df['days_to_expiry'].min()
    max_days = df['days_to_expiry'].max()
    
    # Create regular grid
    strike_grid = np.linspace(min_strike, max_strike, 50)
    days_grid = np.linspace(min_days, max_days, 20)
    X, Y = np.meshgrid(strike_grid, days_grid)
    
    # Interpolate IV values onto the grid
    from scipy.interpolate import griddata
    
    points = df[['strike', 'days_to_expiry']].values
    values = df['iv'].values
    
    Z = griddata(points, values, (X, Y), method='linear', fill_value=np.nan)
    
    # Create comprehensive figure with multiple views
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            'Volatility Surface (3D)', 
            'IV vs Strike (by Expiry)', 
            'Term Structure (ATM)', 
            'Moneyness Surface'
        ),
        specs=[
            [{'type': 'surface'}, {'type': 'scatter'}],
            [{'type': 'scatter'}, {'type': 'surface'}]
        ],
        vertical_spacing=0.15,
        horizontal_spacing=0.1
    )
    
    # 1. Main 3D Volatility Surface
    surface = go.Surface(
        x=X,
        y=Y,
        z=Z,
        colorscale='Viridis',
        name='IV Surface',
        colorbar=dict(
            title="Implied Volatility (%)",
            x=0.45
        ),
        hovertemplate="Strike: %{x:.0f}<br>" +
                    "Days to Expiry: %{y:.1f}<br>" +
                    "IV: %{z:.1f}%<extra></extra>"
    )
    
    fig.add_trace(surface, row=1, col=1)
    
    # 2. IV vs Strike colored by expiry
    colors = px.colors.qualitative.Plotly
    for i, result in enumerate(results):
        color = colors[i % len(colors)]
        fig.add_trace(
            go.Scatter(
                x=result['strikes'],
                y=result['ivs'] * 100,
                mode='markers+lines',
                name=f"{result['expiry_str'][:10]}",
                marker=dict(color=color, size=6),
                line=dict(color=color, width=2),
                hovertemplate=f"Expiry: {result['expiry_str']}<br>" +
                            "Strike: %{x:.0f}<br>" +
                            "IV: %{y:.1f}%<extra></extra>"
            ),
            row=1, col=2
        )
    
    # Add spot line to IV plot
    if results:
        spot = results[0]['spot']
        fig.add_trace(
            go.Scatter(
                x=[spot, spot],
                y=[df['iv'].min(), df['iv'].max()],
                mode='lines',
                line=dict(color='red', dash='dash', width=2),
                name=f'Spot: ${spot:.0f}',
                showlegend=False,
                hovertemplate=f"Spot Price: ${spot:.0f}<extra></extra>"
            ),
            row=1, col=2
        )
    
    # 3. Term Structure (ATM IV)
    atm_data = []
    for result in results:
        spot = result['spot']
        strikes = result['strikes']
        ivs = result['ivs']
        
        # Find ATM IV (closest to spot)
        atm_idx = np.argmin(np.abs(strikes - spot))
        atm_iv = ivs[atm_idx] * 100
        
        days_to_expiry = (result['expiry'] - results[0]['expiry']) / (1000 * 60 * 60 * 24)
        
        atm_data.append({
            'days': days_to_expiry,
            'atm_iv': atm_iv,
            'expiry_str': result['expiry_str'][:10]
        })
    
    atm_df = pd.DataFrame(atm_data)
    
    fig.add_trace(
        go.Scatter(
            x=atm_df['days'],
            y=atm_df['atm_iv'],
            mode='markers+lines+text',
            text=atm_df['expiry_str'],
            textposition='top center',
            marker=dict(size=10, color='blue'),
            line=dict(color='blue', width=3),
            name='ATM IV Term Structure',
            hovertemplate="Days to Expiry: %{x:.1f}<br>" +
                        "ATM IV: %{y:.1f}%<extra></extra>"
        ),
        row=2, col=1
    )
    
    # 4. Moneyness Surface
    # Create moneyness grid
    min_moneyness = df['moneyness'].min()
    max_moneyness = df['moneyness'].max()
    moneyness_grid = np.linspace(min_moneyness, max_moneyness, 50)
    X_money, Y_money = np.meshgrid(moneyness_grid, days_grid)
    
    # Interpolate using moneyness
    points_money = df[['moneyness', 'days_to_expiry']].values
    Z_money = griddata(points_money, values, (X_money, Y_money), method='linear', fill_value=np.nan)
    
    fig.add_trace(
        go.Surface(
            x=X_money,
            y=Y_money,
            z=Z_money,
            colorscale='Plasma',
            name='Moneyness Surface',
            showscale=False,
            hovertemplate="Moneyness: %{x:.2f}<br>" +
                        "Days to Expiry: %{y:.1f}<br>" +
                        "IV: %{z:.1f}%<extra></extra>"
        ),
        row=2, col=2
    )
    
    # Update layout for each subplot
    fig.update_layout(
        height=1200,
        title_text=f"Comprehensive Volatility Surface Analysis - {len(results)} Expiries",
        showlegend=True,
        scene=dict(
            xaxis_title="Strike Price ($)",
            yaxis_title="Days to Expiry",
            zaxis_title="Implied Volatility (%)",
            camera=dict(eye=dict(x=1.5, y=1.5, z=1.5))
        ),
        scene2=dict(
            xaxis_title="Moneyness (K/S)",
            yaxis_title="Days to Expiry", 
            zaxis_title="Implied Volatility (%)",
            camera=dict(eye=dict(x=1.5, y=1.5, z=1.5))
        )
    )
    
    # Update subplot axes
    fig.update_xaxes(title_text="Strike Price ($)", row=1, col=2)
    fig.update_yaxes(title_text="Implied Volatility (%)", row=1, col=2)
    
    fig.update_xaxes(title_text="Days to Expiry", row=2, col=1)
    fig.update_yaxes(title_text="ATM Implied Volatility (%)", row=2, col=1)
    
    # Print surface statistics
    print(f"\n📈 Volatility Surface Statistics:")
    print(f"   Strike range: ${min_strike:.0f} - ${max_strike:.0f}")
    print(f"   Time range: {min_days:.1f} - {max_days:.1f} days")
    print(f"   IV range: {df['iv'].min():.1f}% - {df['iv'].max():.1f}%")
    print(f"   Moneyness range: {min_moneyness:.2f} - {max_moneyness:.2f}")
    
    # Print term structure
    print(f"\n⏰ ATM Term Structure:")
    for _, row in atm_df.iterrows():
        print(f"   {row['expiry_str']}: {row['atm_iv']:.1f}% ({row['days']:.1f} days)")
    
    return fig, df

# Create volatility surface if analysis results exist
if 'analysis_results' in globals():
    vol_surface_fig, surface_df = create_volatility_surface(analysis_results)
    if vol_surface_fig:
        vol_surface_fig.show()
        
        # Save surface data to CSV for further analysis
        surface_df.to_csv('volatility_surface_data.csv', index=False)
        print(f"\n💾 Surface data saved to: volatility_surface_data.csv")
else:
    print("⚠️ No analysis results found. Please run the main analysis cell first.")

🏗️ Constructing volatility surface from 10 expiries...
📊 Surface data: 307 points across 10 expiries

📈 Volatility Surface Statistics:
   Strike range: $20000 - $400000
   Time range: 0.0 - 126.0 days
   IV range: 29.5% - 145.3%
   Moneyness range: 0.18 - 3.53

⏰ ATM Term Structure:
   2025-08-22: 32.7% (0.0 days)
   2025-08-23: 36.6% (1.0 days)
   2025-08-24: 32.3% (2.0 days)
   2025-08-25: 30.8% (3.0 days)
   2025-08-29: 33.8% (7.0 days)
   2025-09-05: 34.2% (14.0 days)
   2025-09-12: 34.2% (21.0 days)
   2025-09-26: 34.2% (35.0 days)
   2025-10-31: 36.0% (70.0 days)
   2025-12-26: 40.3% (126.0 days)



💾 Surface data saved to: volatility_surface_data.csv


# 📚 Volatility Surface Construction - Detailed Methodology

## Overview
A **volatility surface** is a 3D representation showing how implied volatility varies across two dimensions:
- **X-axis**: Strike prices (or moneyness)
- **Y-axis**: Time to expiry
- **Z-axis**: Implied volatility levels

## 🔧 Step-by-Step Construction Process

### 1. **Data Collection & Organization**
```
For each expiry date:
├── Fetch all available call options
├── Extract strike prices and implied volatilities
├── Calculate days to expiry
└── Store in structured format
```

### 2. **Data Transformation**
```python
# Convert raw data into surface-ready format
surface_data = []
for each_expiry:
    for each_strike_iv_pair:
        surface_data.append({
            'strike': strike_price,
            'days_to_expiry': time_dimension,
            'iv': implied_volatility,
            'moneyness': strike/spot_price
        })
```

### 3. **Grid Creation**
```python
# Create regular grids for interpolation
strike_grid = np.linspace(min_strike, max_strike, 50)  # 50 points
time_grid = np.linspace(min_days, max_days, 20)       # 20 points
X, Y = np.meshgrid(strike_grid, time_grid)            # 50x20 = 1000 points
```

### 4. **Surface Interpolation**
```python
# Use scipy.interpolate.griddata for smooth surface
from scipy.interpolate import griddata

# Input: Scattered market data points
points = [(strike₁, time₁), (strike₂, time₂), ...]
values = [iv₁, iv₂, iv₃, ...]

# Output: Regular grid of interpolated values
Z = griddata(points, values, (X, Y), method='linear')
```

## 🎯 Key Technical Concepts

### **1. Interpolation Methods**
- **Linear**: Fast, preserves monotonicity
- **Cubic**: Smooth but can oscillate
- **Nearest**: No smoothing, preserves market data

### **2. Moneyness Normalization**
```
Moneyness = Strike Price / Spot Price
- 1.0 = At-the-money (ATM)
- < 1.0 = In-the-money (ITM)  
- > 1.0 = Out-of-the-money (OTM)
```

### **3. Time Dimension**
```
Days to Expiry = (Expiry Timestamp - Current Time) / (24 * 60 * 60 * 1000)
```

### **4. Surface Quality Checks**
- **No Arbitrage**: Surface should be smooth without sharp discontinuities
- **Boundary Conditions**: Reasonable behavior at extremes
- **Data Density**: Sufficient points for reliable interpolation

## 📊 Multiple Surface Views

### **1. Strike-Based Surface**
- X: Absolute strike prices ($20K, $100K, $200K, etc.)
- Y: Time to expiry (0, 7, 30, 90 days, etc.)
- Z: Implied volatility (30%, 50%, 100%, etc.)

### **2. Moneyness Surface**
- X: Moneyness ratios (0.8, 1.0, 1.2, etc.)
- Y: Time to expiry
- Z: Implied volatility
- **Advantage**: Normalized view, easier to compare across different spot levels

### **3. Term Structure**
- X: Time to expiry
- Y: ATM implied volatility
- **Shows**: How volatility expectations change over time

## ⚙️ Implementation Details

### **Data Filtering**
```python
# Remove outliers and ensure data quality
valid_data = data[
    (data['iv'] > 0.05) &     # Minimum 5% IV
    (data['iv'] < 3.0) &      # Maximum 300% IV
    (data['moneyness'] > 0.5) & # Strike > 50% of spot
    (data['moneyness'] < 2.0)   # Strike < 200% of spot
]
```

### **Surface Smoothing**
```python
# Apply smoothing to reduce noise
from scipy.ndimage import gaussian_filter
Z_smooth = gaussian_filter(Z, sigma=1.0)
```

### **Gap Filling**
```python
# Handle missing data points
Z_filled = griddata(points, values, (X, Y), 
                   method='linear', 
                   fill_value=np.nan)
```

## 🎨 Visualization Components

### **4-Panel Dashboard**
1. **Main 3D Surface**: Interactive surface plot
2. **IV Smiles**: Individual smile curves by expiry
3. **Term Structure**: ATM IV evolution over time
4. **Moneyness View**: Normalized surface

### **Interactive Features**
- **Hover tooltips**: Exact values at each point
- **3D rotation**: Explore from different angles
- **Zoom/Pan**: Focus on specific regions
- **Color mapping**: Visual representation of IV levels

## 🔍 Mathematical Foundation

### **Surface Equation**
```
IV(K, T) = f(Strike, Time)
where:
- K = Strike price
- T = Time to expiry
- IV = Implied volatility
```

### **Interpolation Formula**
```
For point (x, y) on regular grid:
IV(x, y) = Σ wᵢ × IVᵢ
where wᵢ are interpolation weights
```

## 📈 Applications

### **Trading Applications**
- **Relative Value**: Compare options across strikes/expiries
- **Arbitrage Detection**: Identify pricing inconsistencies
- **Risk Management**: Understand volatility exposure

### **Quantitative Applications**
- **Model Calibration**: Fit stochastic volatility models
- **Scenario Analysis**: Stress test portfolios
- **Greeks Calculation**: Delta, gamma, vega surfaces

## 🎯 Quality Metrics

### **Surface Smoothness**
```python
# Calculate surface curvature
d2z_dx2 = np.gradient(np.gradient(Z, axis=1), axis=1)
d2z_dy2 = np.gradient(np.gradient(Z, axis=0), axis=0)
curvature = np.abs(d2z_dx2) + np.abs(d2z_dy2)
```

### **Interpolation Error**
```python
# Cross-validation: Remove points and test interpolation
from sklearn.model_selection import cross_val_score
errors = []
for holdout_point in market_data:
    predicted = interpolate_without_point(holdout_point)
    errors.append(abs(predicted - actual))
```

This comprehensive approach ensures we create a robust, high-quality volatility surface that accurately represents market conditions across all strikes and expiries!

In [7]:
# 🛠️ Practical Example: Building a Volatility Surface Step-by-Step

def demonstrate_surface_construction():
    """
    Step-by-step demonstration of how we build the volatility surface
    from our options data
    """
    
    if 'analysis_results' not in globals():
        print("❌ Please run the main analysis first to generate data!")
        return
    
    print("🔧 STEP-BY-STEP VOLATILITY SURFACE CONSTRUCTION")
    print("=" * 60)
    
    results = analysis_results
    
    # STEP 1: Data Collection and Organization
    print("\n📊 STEP 1: COLLECT & ORGANIZE DATA")
    print("-" * 40)
    
    raw_data_points = []
    for i, result in enumerate(results):
        strikes = result['strikes']
        ivs = result['ivs']
        expiry_str = result['expiry_str']
        days_to_expiry = (result['expiry'] - results[0]['expiry']) / (1000 * 60 * 60 * 24)
        
        print(f"Expiry {i+1}: {expiry_str[:10]}")
        print(f"  └─ {len(strikes)} options, {days_to_expiry:.1f} days out")
        print(f"  └─ Strike range: ${strikes.min():,.0f} - ${strikes.max():,.0f}")
        print(f"  └─ IV range: {ivs.min()*100:.1f}% - {ivs.max()*100:.1f}%")
        
        # Store each individual option as a data point
        for strike, iv in zip(strikes, ivs):
            raw_data_points.append({
                'strike': strike,
                'days_to_expiry': days_to_expiry,
                'iv_percent': iv * 100,
                'expiry_str': expiry_str[:10],
                'moneyness': strike / result['spot']
            })
    
    # Convert to DataFrame
    df_raw = pd.DataFrame(raw_data_points)
    print(f"\n✅ Total data points collected: {len(df_raw)}")
    print(f"   Strike range: ${df_raw['strike'].min():,.0f} - ${df_raw['strike'].max():,.0f}")
    print(f"   Time range: {df_raw['days_to_expiry'].min():.1f} - {df_raw['days_to_expiry'].max():.1f} days")
    print(f"   IV range: {df_raw['iv_percent'].min():.1f}% - {df_raw['iv_percent'].max():.1f}%")
    
    # STEP 2: Create Regular Grids
    print("\n🔲 STEP 2: CREATE INTERPOLATION GRIDS")
    print("-" * 40)
    
    # Strike grid
    min_strike = df_raw['strike'].min()
    max_strike = df_raw['strike'].max()
    strike_points = 30  # Reduced for clarity
    strike_grid = np.linspace(min_strike, max_strike, strike_points)
    
    # Time grid
    min_days = df_raw['days_to_expiry'].min()
    max_days = df_raw['days_to_expiry'].max()
    time_points = 15  # Reduced for clarity
    time_grid = np.linspace(min_days, max_days, time_points)
    
    print(f"Strike grid: {strike_points} points from ${min_strike:,.0f} to ${max_strike:,.0f}")
    print(f"Time grid: {time_points} points from {min_days:.1f} to {max_days:.1f} days")
    print(f"Total grid points: {strike_points} × {time_points} = {strike_points * time_points}")
    
    # Create meshgrid
    X_grid, Y_grid = np.meshgrid(strike_grid, time_grid)
    print(f"Meshgrid shape: {X_grid.shape}")
    
    # STEP 3: Interpolation Process
    print("\n🔄 STEP 3: INTERPOLATE SCATTERED DATA TO REGULAR GRID")
    print("-" * 40)
    
    from scipy.interpolate import griddata
    
    # Prepare input data for interpolation
    points_input = df_raw[['strike', 'days_to_expiry']].values
    values_input = df_raw['iv_percent'].values
    
    print(f"Input: {len(points_input)} scattered market points")
    print(f"Output: {X_grid.size} regular grid points")
    
    # Perform interpolation
    print("Interpolating using linear method...")
    Z_interpolated = griddata(points_input, values_input, (X_grid, Y_grid), method='linear')
    
    # Count valid interpolated points
    valid_points = np.sum(~np.isnan(Z_interpolated))
    print(f"Successfully interpolated: {valid_points}/{X_grid.size} points ({valid_points/X_grid.size*100:.1f}%)")
    
    # STEP 4: Surface Quality Analysis
    print("\n📏 STEP 4: SURFACE QUALITY ANALYSIS")
    print("-" * 40)
    
    # Calculate surface statistics
    iv_range = np.nanmax(Z_interpolated) - np.nanmin(Z_interpolated)
    iv_mean = np.nanmean(Z_interpolated)
    iv_std = np.nanstd(Z_interpolated)
    
    print(f"Surface IV statistics:")
    print(f"  └─ Range: {np.nanmin(Z_interpolated):.1f}% - {np.nanmax(Z_interpolated):.1f}% (span: {iv_range:.1f}%)")
    print(f"  └─ Mean: {iv_mean:.1f}%")
    print(f"  └─ Std Dev: {iv_std:.1f}%")
    
    # Calculate surface smoothness
    grad_x = np.gradient(Z_interpolated, axis=1)
    grad_y = np.gradient(Z_interpolated, axis=0)
    smoothness = np.nanmean(np.sqrt(grad_x**2 + grad_y**2))
    print(f"  └─ Surface smoothness metric: {smoothness:.2f}%/point")
    
    # STEP 5: Create Visualization
    print("\n🎨 STEP 5: CREATE SURFACE VISUALIZATION")
    print("-" * 40)
    
    fig = go.Figure()
    
    # Add the interpolated surface
    surface_trace = go.Surface(
        x=X_grid,
        y=Y_grid, 
        z=Z_interpolated,
        colorscale='Viridis',
        name='Interpolated Surface',
        opacity=0.8,
        colorbar=dict(title="Implied Volatility (%)"),
        hovertemplate="Strike: $%{x:,.0f}<br>" +
                     "Days: %{y:.1f}<br>" +
                     "IV: %{z:.1f}%<extra></extra>"
    )
    fig.add_trace(surface_trace)
    
    # Add original market data points as scatter
    scatter_trace = go.Scatter3d(
        x=df_raw['strike'],
        y=df_raw['days_to_expiry'],
        z=df_raw['iv_percent'],
        mode='markers',
        marker=dict(
            size=4,
            color='red',
            symbol='circle'
        ),
        name='Market Data',
        hovertemplate="Strike: $%{x:,.0f}<br>" +
                     "Days: %{y:.1f}<br>" +
                     "IV: %{z:.1f}%<br>" +
                     "Expiry: %{text}<extra></extra>",
        text=df_raw['expiry_str']
    )
    fig.add_trace(scatter_trace)
    
    # Update layout
    fig.update_layout(
        title="Volatility Surface Construction Demo<br>" + 
              f"<sub>{len(df_raw)} market points → {strike_points}×{time_points} grid</sub>",
        scene=dict(
            xaxis_title="Strike Price ($)",
            yaxis_title="Days to Expiry",
            zaxis_title="Implied Volatility (%)",
            camera=dict(eye=dict(x=1.2, y=1.2, z=0.8))
        ),
        height=700
    )
    
    print("✅ Surface visualization created!")
    print(f"   Red dots: {len(df_raw)} original market data points")
    print(f"   Blue surface: {strike_points}×{time_points} interpolated grid")
    
    # STEP 6: Extract Key Insights
    print("\n💡 STEP 6: EXTRACT KEY INSIGHTS")
    print("-" * 40)
    
    # Term structure analysis
    atm_ivs = []
    spot = results[0]['spot']
    
    for result in results:
        strikes = result['strikes']
        ivs = result['ivs']
        atm_idx = np.argmin(np.abs(strikes - spot))
        atm_iv = ivs[atm_idx] * 100
        days = (result['expiry'] - results[0]['expiry']) / (1000 * 60 * 60 * 24)
        atm_ivs.append((days, atm_iv))
    
    atm_ivs.sort()
    print("ATM Term Structure:")
    for days, iv in atm_ivs:
        print(f"  └─ {days:5.1f} days: {iv:5.1f}% IV")
    
    # Smile analysis
    print(f"\nSmile Characteristics (at spot ${spot:,.0f}):")
    near_expiry = results[0]  # Shortest expiry
    far_expiry = results[-1]  # Longest expiry
    
    print(f"  └─ Near expiry ({near_expiry['expiry_str'][:10]}):")
    print(f"     ├─ IV range: {near_expiry['ivs'].min()*100:.1f}% - {near_expiry['ivs'].max()*100:.1f}%")
    print(f"     └─ Smile skew: {(near_expiry['ivs'].max() - near_expiry['ivs'].min())*100:.1f}%")
    
    print(f"  └─ Far expiry ({far_expiry['expiry_str'][:10]}):")
    print(f"     ├─ IV range: {far_expiry['ivs'].min()*100:.1f}% - {far_expiry['ivs'].max()*100:.1f}%")
    print(f"     └─ Smile skew: {(far_expiry['ivs'].max() - far_expiry['ivs'].min())*100:.1f}%")
    
    print("\n🎯 SURFACE CONSTRUCTION COMPLETE!")
    print("=" * 60)
    
    return fig, df_raw

# Run the demonstration
demo_fig, demo_data = demonstrate_surface_construction()
if demo_fig:
    demo_fig.show()

🔧 STEP-BY-STEP VOLATILITY SURFACE CONSTRUCTION

📊 STEP 1: COLLECT & ORGANIZE DATA
----------------------------------------
Expiry 1: 2025-08-22
  └─ 38 options, 0.0 days out
  └─ Strike range: $90,000 - $165,000
  └─ IV range: 29.9% - 122.1%
Expiry 2: 2025-08-23
  └─ 16 options, 1.0 days out
  └─ Strike range: $106,000 - $124,000
  └─ IV range: 34.7% - 56.1%
Expiry 3: 2025-08-24
  └─ 15 options, 2.0 days out
  └─ Strike range: $104,000 - $122,000
  └─ IV range: 30.9% - 53.6%
Expiry 4: 2025-08-25
  └─ 14 options, 3.0 days out
  └─ Strike range: $106,000 - $122,000
  └─ IV range: 29.5% - 44.9%
Expiry 5: 2025-08-29
  └─ 50 options, 7.0 days out
  └─ Strike range: $40,000 - $190,000
  └─ IV range: 32.6% - 145.3%
Expiry 6: 2025-09-05
  └─ 28 options, 14.0 days out
  └─ Strike range: $80,000 - $140,000
  └─ IV range: 33.0% - 77.0%
Expiry 7: 2025-09-12
  └─ 17 options, 21.0 days out
  └─ Strike range: $100,000 - $130,000
  └─ IV range: 33.4% - 43.2%
Expiry 8: 2025-09-26
  └─ 50 options, 35.0 

# 🧮 Mathematical Foundation & Key Concepts Summary

## 🔬 The Core Mathematical Process

### **1. From Scattered Points to Continuous Surface**

**Input**: Discrete market observations
```
{(Strike₁, Time₁, IV₁), (Strike₂, Time₂, IV₂), ..., (Strikeₙ, Timeₙ, IVₙ)}
```

**Output**: Continuous function
```
IV(K, T) = f(Strike, Time)
```

**Method**: Multivariate interpolation using scipy's `griddata`

### **2. Interpolation Mathematics**

#### **Linear Interpolation (Delaunay Triangulation)**
```
For point P inside triangle ABC:
IV(P) = wₐ·IV(A) + wᵦ·IV(B) + wᶜ·IV(C)

where wₐ + wᵦ + wᶜ = 1 (barycentric coordinates)
```

#### **Grid Construction**
```python
# Create regular meshgrid
X = [K₁, K₂, ..., Kₘ]  # m strike points
Y = [T₁, T₂, ..., Tₙ]  # n time points
Grid = X × Y            # m×n interpolation points
```

### **3. Surface Quality Metrics**

#### **Smoothness Calculation**
```python
# Surface gradient magnitude
∇IV = √[(∂IV/∂K)² + (∂IV/∂T)²]

# Curvature (second derivatives)
κ = |∂²IV/∂K²| + |∂²IV/∂T²|
```

#### **No-Arbitrage Conditions**
```
1. ∂²C/∂K² ≥ 0  (Risk-neutral density must be non-negative)
2. ∂C/∂T ≥ 0   (Time value decreases with time)
3. C(K₁) ≥ C(K₂) for K₁ < K₂  (Call option monotonicity)
```

## 🎯 Why This Approach Works

### **1. Market Data Preservation**
- Original market points remain **exactly** at their observed values
- Interpolation only fills gaps between known data
- No smoothing that might remove important market signals

### **2. Computational Efficiency**
- Regular grid enables fast lookups: O(1) access time
- Vectorized operations for entire surface calculations
- GPU-friendly data structure for advanced computations

### **3. Risk Management Applications**
```python
# Greeks calculation across entire surface
Delta_surface = np.gradient(option_prices, strike_grid, axis=1)
Gamma_surface = np.gradient(Delta_surface, strike_grid, axis=1)
Theta_surface = np.gradient(option_prices, time_grid, axis=0)
Vega_surface = option_prices * 0.01  # 1% vol move
```

## 📊 Surface Interpretation Guide

### **Shape Analysis**

#### **Volatility Smile/Skew**
```
- Symmetric smile: ∂²IV/∂K² > 0 everywhere
- Volatility skew: ∂IV/∂K ≠ 0 (asymmetric)
- Flat profile: ∂IV/∂K ≈ 0 (Black-Scholes world)
```

#### **Term Structure**
```
- Backwardation: ∂IV/∂T < 0 (short-term > long-term vol)
- Contango: ∂IV/∂T > 0 (long-term > short-term vol)
- Flat term: ∂IV/∂T ≈ 0 (constant vol across time)
```

### **Trading Insights**

#### **Relative Value Identification**
```python
# Find overvalued/undervalued options
surface_iv = interpolate_surface(strike, time)
market_iv = get_market_quote(strike, time)
relative_value = market_iv - surface_iv

if relative_value > threshold:
    print("Option appears overvalued")
elif relative_value < -threshold:
    print("Option appears undervalued")
```

#### **Arbitrage Detection**
```python
# Check for violations
arbitrage_flags = []
if np.any(np.diff(call_prices, axis=1) > 0):  # Call spread arbitrage
    arbitrage_flags.append("Call spread violation")
if np.any(call_prices < intrinsic_values):     # Intrinsic value arbitrage
    arbitrage_flags.append("Below intrinsic value")
```

## 🔧 Implementation Best Practices

### **1. Data Quality Control**
```python
def validate_data(strikes, ivs, expiries):
    # Remove outliers
    valid_mask = (ivs > 0.05) & (ivs < 5.0)  # 5% - 500% IV range
    
    # Check for sufficient data density
    min_points_per_expiry = 4
    
    # Verify monotonicity where expected
    # (e.g., call prices decreasing with strike)
    
    return strikes[valid_mask], ivs[valid_mask]
```

### **2. Interpolation Parameter Tuning**
```python
# Grid resolution trade-off
coarse_grid = (20, 10)    # Fast, less accurate
fine_grid = (100, 50)     # Slow, more accurate
optimal_grid = (50, 25)   # Balanced approach

# Boundary handling
fill_value = np.nan       # Conservative (shows gaps)
fill_value = 'extrapolate' # Aggressive (may create artifacts)
```

### **3. Surface Validation**
```python
def validate_surface(surface):
    # Check for NaN values
    nan_percentage = np.sum(np.isnan(surface)) / surface.size
    
    # Verify reasonable IV ranges
    iv_range = (np.nanmin(surface), np.nanmax(surface))
    
    # Calculate surface metrics
    smoothness = calculate_smoothness(surface)
    
    return {
        'nan_percentage': nan_percentage,
        'iv_range': iv_range,
        'smoothness': smoothness
    }
```

This mathematical framework ensures our volatility surface is both **theoretically sound** and **practically useful** for trading and risk management applications! 🚀

In [18]:
import time
from datetime import datetime
from scipy.interpolate import interp1d

def create_risk_neutral_surface(results):
    """Create interactive 3D surface of risk-neutral probability distributions"""
    if not results:
        print("❌ No data available for surface creation")
        return None, None, None
    
    print(f"🌊 Creating 3D Risk-Neutral Probability Surface from {len(results)} expiries...")
    
    # Extract data for all expiries
    all_densities = []
    all_strikes = []
    all_days = []
    peak_evolution = []
    
    for i, result in enumerate(results):
        if 'rn_density' in result and result['rn_density'] is not None:
            strikes = result['K_dense']  # Dense strike grid
            density = result['rn_density']  # Risk-neutral density
            
            # Filter out strikes > $150k
            mask = strikes <= 150000
            strikes = strikes[mask]
            density = density[mask]
            
            if len(strikes) == 0:  # Skip if no strikes remain after filtering
                continue
            
            # Calculate days to expiry from expiry timestamp
            expiry_timestamp = result['expiry'] / 1000  # Convert to seconds
            current_timestamp = time.time()
            days_to_exp = max(0, (expiry_timestamp - current_timestamp) / (24 * 3600))
            
            # Find peak density location
            max_idx = np.argmax(density)
            peak_strike = strikes[max_idx]
            peak_density = density[max_idx]
            peak_evolution.append({
                'expiry': result['expiry_str'], 
                'days': days_to_exp,
                'peak_strike': peak_strike,
                'peak_density': peak_density
            })
            
            print(f"  📊 {result['expiry_str']}: {len(strikes)} density points, {days_to_exp:.1f} days")
            
            all_strikes.append(strikes)
            all_densities.append(density)
            all_days.append(days_to_exp)
    
    if not all_densities:
        print("❌ No valid density data found")
        return None, None, None
    
    # Create uniform grid for surface (filter strikes > $150k)
    min_strike = min(min(strikes) for strikes in all_strikes)
    max_strike = min(150000, max(max(strikes) for strikes in all_strikes))  # Cap at $150k
    strike_grid = np.linspace(min_strike, max_strike, 100)
    
    # Interpolate all densities to common grid
    surface_data = []
    for strikes, density in zip(all_strikes, all_densities):
        interp_func = interp1d(strikes, density, kind='cubic', 
                              bounds_error=False, fill_value=0)
        surface_data.append(interp_func(strike_grid))
    
    # Convert to numpy array for surface plotting
    Z = np.array(surface_data).T  # Transpose for correct orientation
    X, Y = np.meshgrid(all_days, strike_grid)
    
    print(f"✅ Surface grid: {Z.shape[0]} × {Z.shape[1]} = {Z.size} points")
    print(f"📈 Strike range: ${min_strike:,.0f} - ${max_strike:,.0f}")
    print(f"⏰ Time range: {min(all_days):.1f} - {max(all_days):.1f} days")
    print(f"📊 Density range: {Z.min():.6f} - {Z.max():.6f}")
    
    # Create the main 3D surface plot
    fig = go.Figure()
    
    # Add the 3D surface
    fig.add_trace(
        go.Surface(
            x=X, y=Y, z=Z,
            colorscale='Viridis',
            opacity=0.9,
            showscale=True,
            colorbar=dict(title="Probability Density", x=1.05),
            hovertemplate="Days: %{x:.1f}<br>Strike: $%{y:,.0f}<br>Density: %{z:.6f}<extra></extra>"
        )
    )
    
    # Update layout for 3D scene
    fig.update_layout(
        title=f"🌊 Risk-Neutral Probability Surface Evolution<br>" +
              f"<span style='font-size:12px'>Bitcoin Options - {len(results)} Expiries - Surface shows probability density evolution</span>",
        scene=dict(
            xaxis_title="Days to Expiry",
            yaxis_title="Strike Price ($)",
            zaxis_title="Probability Density",
            camera=dict(
                eye=dict(x=1.5, y=1.5, z=1.2)
            )
        ),
        height=700,
        font=dict(size=12)
    )
    
    return fig, {'X': X, 'Y': Y, 'Z': Z}, peak_evolution

def create_2d_density_evolution(results):
    """Create 2D plot showing density evolution over time"""
    if not results:
        return None
        
    fig = go.Figure()
    
    # Add traces for each expiry
    colors = plt.cm.viridis(np.linspace(0, 1, len(results)))
    
    for i, (result, color) in enumerate(zip(results, colors)):
        if 'rn_density' in result and result['rn_density'] is not None:
            strikes = result['K_dense']
            density = result['rn_density']
            
            # Filter out strikes > $150k
            mask = strikes <= 150000
            strikes = strikes[mask]
            density = density[mask]
            
            if len(strikes) == 0:  # Skip if no strikes remain after filtering
                continue
            
            # Calculate days to expiry
            expiry_timestamp = result['expiry'] / 1000
            current_timestamp = time.time()
            days_to_exp = max(0, (expiry_timestamp - current_timestamp) / (24 * 3600))
            
            fig.add_trace(
                go.Scatter(
                    x=strikes,
                    y=density,
                    mode='lines',
                    name=f"{result['expiry_str']} ({days_to_exp:.0f}d)",
                    line=dict(
                        color=f'rgba({int(color[0]*255)},{int(color[1]*255)},{int(color[2]*255)},0.8)', 
                        width=2
                    ),
                    hovertemplate="Strike: $%{x:,.0f}<br>Density: %{y:.6f}<extra></extra>"
                )
            )
    
    # Add spot price line
    if results:
        spot = results[0]['spot']
        fig.add_vline(
            x=spot, 
            line_dash="dash", 
            line_color="red", 
            annotation_text=f"Current Spot: ${spot:,.0f}",
            annotation_position="top"
        )
    
    fig.update_layout(
        title="📈 Risk-Neutral Probability Density Evolution",
        xaxis_title="Strike Price ($)",
        yaxis_title="Probability Density",
        height=500,
        showlegend=True,
        legend=dict(orientation="v", yanchor="top", y=1, xanchor="left", x=1.02)
    )
    
    return fig

# Create and display both visualizations
if 'analysis_results' in globals():
    print("🌊 Creating Risk-Neutral Probability Visualizations...")
    
    # 3D Surface
    rn_surface_fig, density_surface, peak_evolution = create_risk_neutral_surface(analysis_results)
    
    if rn_surface_fig:
        rn_surface_fig.show()
        
        print("\n📊 Surface Analysis Summary:")
        print(f"   • Grid Resolution: {density_surface['Z'].shape if density_surface else 'N/A'}")
        print(f"   • Peak Migration: {len(peak_evolution)} data points" if peak_evolution else "   • Peak Migration: No data")
        print(f"   • Density Range: {density_surface['Z'].min():.6f} - {density_surface['Z'].max():.6f}" if density_surface else "   • Density Range: No data")
    
    # 2D Evolution Plot
    print("\n📈 Creating 2D density evolution plot...")
    rn_2d_fig = create_2d_density_evolution(analysis_results)
    if rn_2d_fig:
        rn_2d_fig.show()
    
    print("\n✅ Risk-neutral probability visualizations complete!")
else:
    print("❌ analysis_results not found")

🌊 Creating Risk-Neutral Probability Visualizations...
🌊 Creating 3D Risk-Neutral Probability Surface from 10 expiries...
  📊 2025-08-22 10:00:00: 160 density points, 0.7 days
  📊 2025-08-23 10:00:00: 200 density points, 1.7 days
  📊 2025-08-24 10:00:00: 200 density points, 2.7 days
  📊 2025-08-25 10:00:00: 200 density points, 3.7 days
  📊 2025-08-29 10:00:00: 146 density points, 7.7 days
  📊 2025-09-05 10:00:00: 200 density points, 14.7 days
  📊 2025-09-12 10:00:00: 200 density points, 21.7 days
  📊 2025-09-26 10:00:00: 69 density points, 35.7 days
  📊 2025-10-31 09:00:00: 129 density points, 70.7 days
  📊 2025-12-26 09:00:00: 69 density points, 126.7 days
✅ Surface grid: 100 × 10 = 1000 points
📈 Strike range: $20,000 - $149,925
⏰ Time range: 0.7 - 126.7 days
📊 Density range: -0.000009 - 0.000199



📊 Surface Analysis Summary:
   • Grid Resolution: (100, 10)
   • Peak Migration: 10 data points
   • Density Range: -0.000009 - 0.000199

📈 Creating 2D density evolution plot...



✅ Risk-neutral probability visualizations complete!


In [20]:
# Statistical Analysis of Risk-Neutral Distributions
def analyze_risk_neutral_statistics(results):
    """Compute detailed statistics for risk-neutral distributions"""
    
    if not results:
        print("❌ No data available for analysis")
        return None
    
    print("📊 Statistical Analysis of Risk-Neutral Probability Distributions")
    print("=" * 80)
    
    stats_data = []
    spot_price = results[0]['spot']
    
    for result in results:
        if 'rn_density' in result and result['rn_density'] is not None:
            strikes = result['K_dense']
            density = result['rn_density']
            
            # Filter out strikes > $150k
            mask = strikes <= 150000
            strikes = strikes[mask]
            density = density[mask]
            
            if len(strikes) == 0:  # Skip if no strikes remain after filtering
                continue
            
            # Calculate days to expiry
            expiry_timestamp = result['expiry'] / 1000
            current_timestamp = time.time()
            days_to_exp = max(0, (expiry_timestamp - current_timestamp) / (24 * 3600))
            
            # Compute statistics
            # Expected value (first moment)
            expected_price = np.trapezoid(strikes * density, strikes)
            
            # Variance (second central moment)
            variance = np.trapezoid((strikes - expected_price)**2 * density, strikes)
            volatility = np.sqrt(variance)
            
            # Skewness (third standardized moment)
            skewness = np.trapezoid(((strikes - expected_price) / volatility)**3 * density, strikes)
            
            # Kurtosis (fourth standardized moment)
            kurtosis = np.trapezoid(((strikes - expected_price) / volatility)**4 * density, strikes)
            
            # Mode (peak of distribution)
            mode_idx = np.argmax(density)
            mode_price = strikes[mode_idx]
            
            # 95% Confidence interval
            cumulative = np.cumsum(density) * (strikes[1] - strikes[0])
            
            # Handle case where distribution is truncated
            lower_indices = np.where(cumulative >= 0.025)[0]
            upper_indices = np.where(cumulative >= 0.975)[0]
            
            if len(lower_indices) > 0:
                lower_2_5_idx = lower_indices[0]
                ci_lower = strikes[lower_2_5_idx]
            else:
                ci_lower = strikes[0]  # Use minimum strike
                
            if len(upper_indices) > 0:
                upper_97_5_idx = upper_indices[0]
                ci_upper = strikes[upper_97_5_idx]
            else:
                ci_upper = strikes[-1]  # Use maximum strike (capped at $150k)
                
            ci_width = ci_upper - ci_lower
            
            stats = {
                'expiry': result['expiry_str'],
                'days_to_exp': days_to_exp,
                'expected_price': expected_price,
                'mode_price': mode_price,
                'volatility': volatility,
                'skewness': skewness,
                'kurtosis': kurtosis,
                'ci_lower': ci_lower,
                'ci_upper': ci_upper,
                'ci_width': ci_width
            }
            stats_data.append(stats)
            
            print(f"\n🗓️  {result['expiry_str']} ({days_to_exp:.1f} days)")
            print(f"   💰 Expected Price: ${expected_price:,.0f} (vs Spot: ${spot_price:,.0f})")
            print(f"   🎯 Mode (Peak): ${mode_price:,.0f}")
            print(f"   📊 Volatility: ${volatility:,.0f}")
            print(f"   ⚖️  Skewness: {skewness:.3f}")
            print(f"   📈 Kurtosis: {kurtosis:.3f}")
            print(f"   🔀 95% CI: ${ci_lower:,.0f} - ${ci_upper:,.0f} (width: ${ci_width:,.0f})")
    
    if stats_data:
        # Create summary DataFrame
        df_stats = pd.DataFrame(stats_data)
        
        print(f"\n📈 Summary Trends:")
        print(f"   • Expected prices range: ${df_stats['expected_price'].min():,.0f} - ${df_stats['expected_price'].max():,.0f}")
        print(f"   • Average skewness: {df_stats['skewness'].mean():.3f}")
        print(f"   • Average kurtosis: {df_stats['kurtosis'].mean():.3f}")
        print(f"   • Volatility trend: ${df_stats['volatility'].iloc[0]:,.0f} → ${df_stats['volatility'].iloc[-1]:,.0f}")
        
        return df_stats
    
    return None

# Run statistical analysis
if 'analysis_results' in globals():
    stats_df = analyze_risk_neutral_statistics(analysis_results)
    
    if stats_df is not None:
        print(f"\n✅ Statistical analysis complete for {len(stats_df)} expiries")
    else:
        print("❌ Failed to compute statistics")
else:
    print("❌ analysis_results not found")

📊 Statistical Analysis of Risk-Neutral Probability Distributions

🗓️  2025-08-22 10:00:00 (0.7 days)
   💰 Expected Price: $113,725 (vs Spot: $113,455)
   🎯 Mode (Peak): $117,889
   📊 Volatility: $9,048
   ⚖️  Skewness: 0.638
   📈 Kurtosis: 4.841
   🔀 95% CI: $94,146 - $136,357 (width: $42,211)

🗓️  2025-08-23 10:00:00 (1.7 days)
   💰 Expected Price: $117,094 (vs Spot: $113,455)
   🎯 Mode (Peak): $118,030
   📊 Volatility: $3,920
   ⚖️  Skewness: -0.125
   📈 Kurtosis: 1.989
   🔀 95% CI: $109,166 - $123,186 (width: $14,020)

🗓️  2025-08-24 10:00:00 (2.7 days)
   💰 Expected Price: $114,851 (vs Spot: $113,455)
   🎯 Mode (Peak): $118,020
   📊 Volatility: $3,073
   ⚖️  Skewness: -0.535
   📈 Kurtosis: 2.163
   🔀 95% CI: $108,432 - $118,653 (width: $10,221)

🗓️  2025-08-25 10:00:00 (3.7 days)
   💰 Expected Price: $114,400 (vs Spot: $113,455)
   🎯 Mode (Peak): $114,040
   📊 Volatility: $3,260
   ⚖️  Skewness: -0.297
   📈 Kurtosis: 2.768
   🔀 95% CI: $107,367 - $120,472 (width: $13,106)

🗓️  2025

In [17]:
# 🎯 COMPREHENSIVE ANALYSIS SUMMARY
print("🎯 RISK-NEUTRAL PROBABILITY ANALYSIS - KEY INSIGHTS")
print("=" * 80)

current_spot = 113455.47

print(f"\n💰 PRICING INSIGHTS:")
print(f"   • Current Bitcoin Spot Price: ${current_spot:,.0f}")
print(f"   • Near-term (0-4 days) implied expectations: ${stats_df['expected_price'].iloc[:4].mean():,.0f}")
print(f"   • Long-term (70+ days) implied expectations: ${stats_df['expected_price'].iloc[-2:].mean():,.0f}")

print(f"\n📊 MARKET SENTIMENT:")
print(f"   • Short-term volatility (next week): ${stats_df['volatility'].iloc[:5].mean():,.0f}")
print(f"   • Long-term volatility (2+ months): ${stats_df['volatility'].iloc[-3:].mean():,.0f}")

print(f"\n⚖️ RISK SKEW ANALYSIS:")
near_term_skew = stats_df['skewness'].iloc[:5].mean()
long_term_skew = stats_df['skewness'].iloc[-3:].mean()
print(f"   • Near-term skewness: {near_term_skew:.3f} ({'Left-skewed (put bias)' if near_term_skew < 0 else 'Right-skewed (call bias)'})")
print(f"   • Long-term skewness: {long_term_skew:.3f} ({'Left-skewed (put bias)' if long_term_skew < 0 else 'Right-skewed (call bias)'})")

print(f"\n🎢 TAIL RISK (Kurtosis):")
avg_kurtosis = stats_df['kurtosis'].mean()
print(f"   • Average kurtosis: {avg_kurtosis:.2f} ({'High tail risk' if avg_kurtosis > 3 else 'Normal tail risk'})")
print(f"   • Maximum kurtosis: {stats_df['kurtosis'].max():.2f} at {stats_df.loc[stats_df['kurtosis'].idxmax(), 'expiry']}")

print(f"\n🔀 CONFIDENCE INTERVALS:")
print(f"   • Narrowest 95% CI: ${stats_df['ci_width'].min():,.0f} at {stats_df.loc[stats_df['ci_width'].idxmin(), 'expiry']}")
print(f"   • Widest 95% CI: ${stats_df['ci_width'].max():,.0f} at {stats_df.loc[stats_df['ci_width'].idxmax(), 'expiry']}")

print(f"\n🚨 TRADING IMPLICATIONS:")
print(f"   • Time decay is most pronounced in first week (volatility drops from ${stats_df['volatility'].iloc[0]:,.0f} to ${stats_df['volatility'].iloc[4]:,.0f})")
if near_term_skew < 0:
    print(f"   • Near-term put protection is expensive (negative skewness)")
else:
    print(f"   • Near-term upside calls are expensive (positive skewness)")

if avg_kurtosis > 3:
    print(f"   • Fat tails suggest higher probability of extreme moves than normal distribution")
    
print(f"\n📈 PROBABILITY SURFACE CHARACTERISTICS:")
print(f"   • Surface shows {len(analysis_results)} expiration dates")
print(f"   • Strike range: $20K - $400K")
print(f"   • Peak density migration visible across time dimension")
print(f"   • Density values range from {density_surface['Z'].min():.6f} to {density_surface['Z'].max():.6f}")

print(f"\n✅ ANALYSIS COMPLETE!")
print(f"   📊 Processed {len(analysis_results)} option expiries")
print(f"   🌊 Generated 3D probability surface with {density_surface['Z'].size:,} data points")
print(f"   📈 Created 2D evolution plots for temporal analysis")
print(f"   📋 Computed comprehensive statistics for all distributions")

🎯 RISK-NEUTRAL PROBABILITY ANALYSIS - KEY INSIGHTS

💰 PRICING INSIGHTS:
   • Current Bitcoin Spot Price: $113,455
   • Near-term (0-4 days) implied expectations: $115,390
   • Long-term (70+ days) implied expectations: $113,651

📊 MARKET SENTIMENT:
   • Short-term volatility (next week): $8,586
   • Long-term volatility (2+ months): $25,814

⚖️ RISK SKEW ANALYSIS:
   • Near-term skewness: -0.230 (Left-skewed (put bias))
   • Long-term skewness: 0.552 (Right-skewed (call bias))

🎢 TAIL RISK (Kurtosis):
   • Average kurtosis: 4.75 (High tail risk)
   • Maximum kurtosis: 11.14 at 2025-09-26 10:00:00

🔀 CONFIDENCE INTERVALS:
   • Narrowest 95% CI: $10,221 at 2025-08-24 10:00:00
   • Widest 95% CI: $131,759 at 2025-09-26 10:00:00

🚨 TRADING IMPLICATIONS:
   • Time decay is most pronounced in first week (volatility drops from $9,903 to $22,772)
   • Near-term put protection is expensive (negative skewness)
   • Fat tails suggest higher probability of extreme moves than normal distribution

📈

In [10]:
# Debug: Check the structure of analysis_results
if 'analysis_results' in globals():
    print("🔍 Debugging analysis_results structure:")
    print(f"   • Type: {type(analysis_results)}")
    print(f"   • Length: {len(analysis_results) if hasattr(analysis_results, '__len__') else 'No length'}")
    
    if hasattr(analysis_results, '__len__') and len(analysis_results) > 0:
        print(f"\n📋 First result keys: {list(analysis_results[0].keys()) if isinstance(analysis_results[0], dict) else 'Not a dict'}")
        
        # Check if density data exists in different structure
        first_result = analysis_results[0]
        if isinstance(first_result, dict):
            print(f"\n🔍 First result details:")
            for key, value in first_result.items():
                if isinstance(value, dict):
                    print(f"   • {key}: dict with keys {list(value.keys())}")
                elif isinstance(value, (list, np.ndarray)):
                    print(f"   • {key}: {type(value).__name__} with length {len(value)}")
                else:
                    print(f"   • {key}: {type(value).__name__} = {value}")
else:
    print("❌ analysis_results not found in globals")

🔍 Debugging analysis_results structure:
   • Type: <class 'list'>
   • Length: 10

📋 First result keys: ['expiry', 'expiry_str', 'strikes', 'ivs', 'K_dense', 'rn_density', 'spot']

🔍 First result details:
   • expiry: int = 1755849600000
   • expiry_str: str = 2025-08-22 10:00:00
   • strikes: ndarray with length 38
   • ivs: ndarray with length 38
   • K_dense: ndarray with length 200
   • rn_density: ndarray with length 200
   • spot: float = 113455.47
